In [1]:
import tensorflow as tf 
import os 
import PIL
import pandas as pd 
import numpy as np 
import pathlib
import matplotlib.pyplot as plt 
import cv2
import warnings
import sys
import re


In [2]:
# To give access to automl files
sys.path.append("/kaggle/input/automl")
sys.path.append("/kaggle/input/automl/automl")
sys.path.append("/kaggle/input/automl/automl/efficientnetv2")
# sys.path.append("/kaggle/input/automl/automl/efficientnetv2/effnetv2_model.py")
import effnetv2_model

In [2]:
data_dir = pathlib.Path('../input/chexpert-dataset/CheXpert-v1.0-small') 
os.listdir(data_dir)

['valid.csv', 'valid', 'train.csv', 'train']

In [3]:
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
valid_df = pd.read_csv(os.path.join(data_dir, 'valid.csv'))

In [4]:
# Replacing all the NaN values in train and validation dataframes with 0.0 for the 14 labels
label_columns = list(train_df.iloc[:5, 5:].columns)
train_df.loc[:, label_columns] = train_df.loc[:, label_columns].replace(to_replace = np.nan, value = 0.0)

valid_df.loc[:, label_columns] = valid_df.loc[:, label_columns].replace(to_replace = np.nan, value= 0.0)

In [5]:
# Replacing uncertain labels with positive labels
def uncertain_to_pos(df):
    df.replace(to_replace = -1.0, value = 1.0, inplace = True)

uncertain_to_pos(train_df)
uncertain_to_pos(valid_df)

In [6]:
train_df.set_index('Path', inplace = True)
valid_df.set_index('Path', inplace = True)

In [7]:
main_df = pd.concat([train_df, valid_df], axis = 0)

In [8]:
sample_path = [str(next(pathlib.Path(os.path.join(data_dir, 'train')).glob('*/*/*')))]
sample_path

['../input/chexpert-dataset/CheXpert-v1.0-small/train/patient00734/study3/view2_lateral.jpg']

In [38]:
with open('../input/paths-text-files/chexpert_train_image_paths.txt', 'r') as file:
    train_image_paths = file.readline()
with open('../input/paths-text-files/chexpert_valid_image_paths.txt', 'r') as file:
    valid_image_paths = file.readline()

In [39]:
import ast
train_image_paths = ast.literal_eval(train_image_paths)
valid_image_paths = ast.literal_eval(valid_image_paths)

In [40]:
import random
random.seed(42)
random.shuffle(train_image_paths)

In [41]:
train_image_paths[:10]

['../input/chexpert-dataset/CheXpert-v1.0-small/train/patient42951/study1/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient00116/study1/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient05973/study3/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient22599/study4/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient54162/study3/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient20445/study14/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient44848/study1/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient26785/study13/view1_frontal.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient27824/study1/view2_lateral.jpg',
 '../input/chexpert-dataset/CheXpert-v1.0-small/train/patient15462/study2/view2_lateral.jpg']

In [42]:
train_image_paths.remove('../input/chexpert-dataset/CheXpert-v1.0-small/train/patient00001/study1/._view1_frontal.jpg')

In [15]:
# image_count = tf.data.experimental.cardinality(train_ds).numpy()

# val_size = int(image_count * 0.5)
# train_final_ds = train_ds.skip(val_size)
# val_ds = train_ds.take(val_size)

In [43]:
def process_path(file_path):
    parts = list(tf.strings.split(file_path, os.path.sep).numpy()[-5:])
    parts = [i.decode() for i in parts]
    min_path = '/'.join(parts)
    label = tf.cast(list(main_df.loc[min_path,label_columns]), dtype = tf.int16)
    
    # Loading the image
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels = 1)
    # Resizing the image
#     img = tf.image.resize(img, [600,600])
#     img = tf.image.grayscale_to_rgb(img) 
    img = tf.image.convert_image_dtype(img, dtype=tf.uint8)
    img = tf.io.encode_jpeg(img)
    return img, label

In [44]:
# train_ds = tf.data.Dataset.list_files(str(data_dir/'train/*/*/*'), shuffle=True)
# test_ds = tf.data.Dataset.list_files(str(data_dir/'valid/*/*/*'), shuffle=True)
train_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
test_ds = tf.data.Dataset.from_tensor_slices(valid_image_paths)

In [45]:
train_final_ds = train_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.string, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
#valid_ds = val_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.string, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)

In [46]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

# def process_image(image, label):
#     image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
#     image = tf.io.encode_jpeg(image)
#     return image, label

# ds_train_encoded = (
#     ds_train
#     .unbatch()
#     .map(process_image)
# )

# ds_valid_encoded = (
#     ds_valid
#     .unbatch()
#     .map(process_image)
# )

In [47]:
def make_example(encoded_image, label):
    image_feature = Feature(
        bytes_list=BytesList(value=[
            encoded_image,
        ]),
    )
    label_feature = Feature(
        int64_list=Int64List(value=[
            *label,
        ])
    )

    features = Features(feature={
        'image': image_feature,
        'label': label_feature,
    })
    
    example = Example(features=features)
    
    return example.SerializeToString()

In [ ]:
!mkdir '/kaggle/working/training'

NUM_SHARDS = 32
PATH = '/kaggle/working/training/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        train_final_ds
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

mkdir: cannot create directory ‘/kaggle/working/training’: File exists


In [71]:
!mkdir '/kaggle/working/validation'

NUM_SHARDS = 8
PATH = '/kaggle/working/validation/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

SyntaxError: invalid syntax (<ipython-input-71-26301d86f6b3>, line 9)

In [ ]:
batch_size = 8
train = train_final_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid = valid_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Learning Rate scheduler
learning_rate_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0001, 
                                              decay_steps=1000,
                                              decay_rate = 0.8,
                                             )

In [ ]:
input_height, input_width = (600,600)

In [ ]:
tf.keras.backend.clear_session()
effnetv2_s = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[input_height,input_width,3]),
    effnetv2_model.get_model('efficientnetv2-s', include_top=False, pretrained=False),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(14, activation='sigmoid'),
])
effnetv2_s.summary()

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate_scheduler,)

effnetv2_s.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.AUC()])

In [ ]:
os.listdir('..')

In [ ]:
checkpoint_path = 'checkpoints/train/efficientNetV2-s/600px'

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(min_delta = 0.001, patience = 3, restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, store_best_only = True)
callbacks = [early_stopping, checkpoint]

In [ ]:
effnetv2_s.fit(train,validation_data=test, epochs = 10, callbacks=callbacks, verbose = 1)

- Training the model with learning rate 0.01 did not result in faster learning instead it slowed down as the updates were quite big resulting in heavy zig zag movement. This shows that using a large learning rate harms the training speed for chexpert dataset.

In [ ]:
effnetv2_s.save_weights('efficientNetV2_s_600px/weights')